In [1]:
import os
from os.path import isfile
import sys
import glob

os.environ["CUDA_VISIBLE_DEVICES"]="1"
#os.environ["TF_FORCE_GPU_ALLOW_GROWTH"]="true"
os.environ["TF_GPU_ALLOCATOR"]="cuda_malloc_async"

import numpy as np
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Dense, Concatenate, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1, L2
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.preprocessing import MinMaxScaler

import mlflow
import mlflow.tensorflow

2024-09-05 08:32:22.213689: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 08:32:22.320068: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
mlflow.tensorflow.autolog()
mlflow.set_experiment("HSEL fixed q Reduced all levels new")

2024/09/05 08:32:27 INFO mlflow.tracking.fluent: Experiment with name 'HSEL fixed q Reduced all levels new' does not exist. Creating a new experiment.


<Experiment: artifact_location='file:///data/nature_run/work/src/mlruns/931405904624679090', creation_time=1725539547129, experiment_id='931405904624679090', last_update_time=1725539547129, lifecycle_stage='active', name='HSEL fixed q Reduced all levels new', tags={}>

In [ ]:
#train = ['/data/nature_run/fulldays_reduced/all_20060803.npz', '/data/nature_run/fulldays_reduced/all_20060815.npz', '/data/nature_run/fulldays_reduced/all_20060915.npz']
#test = ['/data/nature_run/fulldays_reduced/all_20060315.npz', '/data/nature_run/fulldays_reduced/all_20060515.npz', '/data/nature_run/fulldays_reduced/all_20060615.npz', '/data/nature_run/fulldays_reduced/all_20060715.npz', '/data/nature_run/fulldays_reduced/all_20061015.npz', '/data/nature_run/fulldays_reduced/all_20061115.npz', '/data/nature_run/fulldays_reduced/all_20061215.npz']

#print(len(train), len(test), len(train)+ len(test)) 


In [ ]:
files = glob.glob("/data/nature_run/fulldays_reduced_evenmore/*.npz")
print(files)

In [ ]:
def get_data(filelist):
    hsel_val = []
    scalar_val = []
    table_val = []

    for i in filelist:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        hsel_val.append(tmp["hsel"])
        scalar_val.append(tmp["scalar"])
        table_val.append(tmp["table"])

    return np.vstack(hsel_val), np.vstack(scalar_val), np.vstack(table_val)


train = ['/data/nature_run/fulldays_reduced_evenmore/all_20060815.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060915.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20061015.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060515.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060715.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20061115.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060315.npz', 
         '/data/nature_run/fulldays_reduced_evenmore/all_20061215.npz',
         '/data/nature_run/fulldays_reduced_evenmore/all_20060615.npz']

test = ['/data/nature_run/fulldays_reduced_evenmore/all_20060803.npz']

validation = ['/data/nature_run/fulldays_reduced_evenmore/all_20060803.npz']

hsel_train, scalar_train, table_train = get_data(train)
hsel_test, scalar_test, table_test = get_data(test)
hsel_val, scalar_val, table_val = get_data(validation)

In [4]:
#AE

def dataload(instr):

    train = ["/data/nature_run/fulldays_reduced_evenmore/20060615/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20061215/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060515/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060815/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060915/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060715/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20061015/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20060315/mh.npy",
             "/data/nature_run/fulldays_reduced_evenmore/20061115/mh.npy"]

    test = ["/data/nature_run/fulldays_reduced_evenmore/20060803/mh.npy"]

    if instr == "hsel":
        train = [x.replace("mh", instr) for x in train]
        test = [x.replace("mh", instr) for x in test]

    dat_train = []
    scalar_train = []
    table_train = []
    
    dat_test = []
    scalar_test = []
    table_test = []
    
    for i in train:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        dat_train.append(tmp)
        
        tmp = np.load(i.replace(instr, "table"))
        table_train.append(tmp)
    
        tmp = np.load(i.replace(instr, "scalar"))
        scalar_train.append(tmp)
    
    
    dat_train = np.vstack(dat_train)
    table_train = np.vstack(table_train)
    scalar_train = np.vstack(scalar_train)
        
    for i in test:
        # Keys: 'mh', 'hsel', 'scalar', 'table']
        tmp = np.load(i)
        dat_test.append(tmp)
        
        tmp = np.load(i.replace(instr, "table"))
        table_test.append(tmp)
    
        tmp = np.load(i.replace(instr, "scalar"))
        scalar_test.append(tmp)
    
    
    dat_test = np.vstack(dat_test)
    table_test = np.vstack(table_test)
    scalar_test = np.vstack(scalar_test)
    
    return dat_train, table_train, scalar_train, dat_test, table_test, scalar_test


In [6]:
hsel_train, table_train, scalar_train, hsel_test, table_test, scalar_test = dataload("hsel")
hsel_val = hsel_test.copy()
table_val = table_test.copy()
scalar_val = scalar_test.copy()


In [ ]:
# spress_test = scalar_test[:, 3].reshape(-1, 1)
# spress_train = scalar_train[:, 3].reshape(-1, 1)
# spress_val = scalar_val[:, 3].reshape(-1, 1)


# spress_test
# spress_train
# spress_val

# q_train = table_train[:, :, 1]
# q_test = table_test[:, :, 1]
# q_val = table_val[:, :, 1]

# mins = np.min(hsel_val, axis=0)
# maxs = np.max(hsel_val, axis=0)
# hsel_train = (hsel_val - mins)/(maxs - mins)
# np.savez("hsel_val_scalar.npz", mins=mins, maxs=maxs)

# mins = np.min(hsel_train, axis=0)
# maxs = np.max(hsel_train, axis=0)
# hsel_train = (hsel_train - mins)/(maxs - mins)
# np.savez("hsel_train_scalar.npz", mins=mins, maxs=maxs)

# mins = np.min(hsel_test, axis=0)
# maxs = np.max(hsel_test, axis=0)
# hsel_test = (hsel_test - mins)/(maxs - mins)
# np.savez("hsel_test_scalar.npz", mins=mins, maxs=maxs)

In [7]:
# What should we do
# 1) Only use train scalars
# 2) Scale seperateley


# Presssure
spress_test = scalar_test[:, 3].reshape(-1, 1)
spress_train = scalar_train[:, 3].reshape(-1, 1)
spress_val = scalar_val[:, 3].reshape(-1, 1)

mins = np.min(spress_train, axis=0)
maxs = np.max(spress_train, axis=0)
np.savez("spress_scalar.npz", mins=mins, maxs=maxs)
spress_test = (spress_test - mins)/(maxs - mins)
spress_train = (spress_train - mins)/(maxs - mins)
spress_val = (spress_val - mins)/(maxs - mins)


# Labels
q_train = table_train[:, :, 2]
q_test = table_test[:, :, 2]
q_val = table_val[:, :, 2]


# Spectra

#mins = np.min(hsel_train, axis=0)
#maxs = np.max(hsel_train, axis=0)

s_factors = np.load("minimac_scaling_factors_hsel.npz")
mins = s_factors['mins']
maxs = s_factors['maxs']


hsel_train = np.nan_to_num(hsel_train)
hsel_test = np.nan_to_num(hsel_test)
hsel_val = np.nan_to_num(hsel_val)

hsel_train = (hsel_train - mins)/(maxs - mins)
hsel_test = (hsel_test - mins)/(maxs - mins)
hsel_val = (hsel_val - mins)/(maxs - mins)


with tf.device('/cpu:0'):
    # train
    x_train = {'rad': tf.convert_to_tensor(hsel_train, np.float32),
               'spress': tf.convert_to_tensor(spress_train, np.float32)}
    del hsel_train
    del spress_train
    
    y_train =  tf.convert_to_tensor(q_train, np.float32)
    del q_train

    
    # Val
    x_val = {'rad': tf.convert_to_tensor(hsel_val, np.float32),
             'spress': tf.convert_to_tensor(spress_val, np.float32)}
    del hsel_val
    del spress_val
    
    y_val =  tf.convert_to_tensor(q_val, np.float32)
    del q_val

    
    # Test
    x_test = {'rad': tf.convert_to_tensor(hsel_test, np.float32),
              'spress': tf.convert_to_tensor(spress_test, np.float32)}
    del hsel_test
    del spress_test
    
    y_test =  tf.convert_to_tensor(q_test, np.float32)
    del q_test


2024-09-05 08:51:50.321848: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-05 08:51:50.852603: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2024-09-05 08:51:50.853454: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1616] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 31017 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB, pci bus id: 0000:1b:00.0, compute capability: 7.0


In [ ]:
# print(x_test)
# print(x_val)
# print(y_train.shape)
#a = np.argwhere(np.isnan(x_train['rad']))
#print(np.sum(np.isnan(x_train['rad'])))
#np.nan_to_num(x_train['rad'], copy=False)
#print(np.sum(np.isnan(x_train['rad'])))
#print(a)
#print(len(a))

np.sum(np.isnan(x_train['rad']))

In [8]:
def build_ae_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    # [ha, hb, hc, hd, hw, mh]

    encoder = load_model("encoder_3_mae.keras")
    encoder.trainable = False
    #for layer in encoder.layers:
    #    layer.trainable = False

    
    enc = encoder(mh)
    x = Concatenate()([enc, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="Temp")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 1957,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_ae_model(config)
model.summary()


Model: "Temp"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rad (InputLayer)               [(None, 1957)]       0           []                               
                                                                                                  
 model_1 (Functional)           (None, 64)           275468      ['rad[0][0]']                    
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 concatenate (Concatenate)      (None, 65)           0           ['model_1[0][0]',                
                                                                  'spress[0][0]']              

In [9]:
def build_model(config):
    # ATMS 22
    mh = Input(shape=(config["shape"],), name="rad")
    spress = Input(shape=(1,), name="spress")
    # [ha, hb, hc, hd, hw, mh]
    
    x = Concatenate()([mh, spress])
    for i in range(config["num_layers"]):
        x =  Dropout(config["dropout"])(Dense(config["num_neurons"], 
                                             activation=config["activation"])(x))
    outputs = Dense(config['output'], name="Temp")(x)

    model = Model(inputs=[mh, spress], outputs=outputs, name="Temp")
    model.compile(optimizer="adam", loss='mae')

    return model

config = {'shape': 1957,
          'output': 72,
          'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
          'num_layers': np.random.choice([3, 4, 5, 10]),
          'num_neurons': np.random.choice([16, 32, 64, 128]),
          'activation': np.random.choice(['gelu', 'gelu', 'relu'])}

model = build_model(config)
model.summary()


Model: "Temp"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rad (InputLayer)               [(None, 1957)]       0           []                               
                                                                                                  
 spress (InputLayer)            [(None, 1)]          0           []                               
                                                                                                  
 concatenate_1 (Concatenate)    (None, 1958)         0           ['rad[0][0]',                    
                                                                  'spress[0][0]']                 
                                                                                                  
 dense_4 (Dense)                (None, 64)           125376      ['concatenate_1[0][0]']       

In [10]:
print("DING")

DING


In [ ]:
# AE

for _ in range(100):
    config = {'shape': 1957,
              'output': 72,
              'dropout': np.random.choice([0.01, 0.1, 0.2, 0.5]),
              'num_layers': np.random.choice([3, 4, 5, 10]),
              'num_neurons': np.random.choice([16, 32, 64, 128]),
              'activation': np.random.choice(['gelu', 'gelu', 'relu'])}
    
    ae = np.random.choice([True, False, True, True])
    if ae:
        model = build_ae_model(config)
    else:
        model = build_model(config)
    
    with mlflow.start_run() as run:
        model_checkpoint_callback = ModelCheckpoint(
            filepath=f"models/model_{run.info.run_id}.keras",
            save_weights_only=False,
            monitor='val_loss',
            mode='min',
            save_best_only=True)
        callback = EarlyStopping(monitor='val_loss', patience=12, verbose=1)
        mlflow.log_param("Autoencoder", ae)

        history = model.fit(x_train, y_train, epochs=1000, batch_size=2000,
                            validation_data=(x_val, y_val),
                            callbacks=[callback, model_checkpoint_callback])
        
        evaled_test = model.evaluate(x_train, y_train, batch_size=100)
        evaled_train = model.evaluate(x_test, y_test, batch_size=100)

        mlflow.log_metric("test_loss", evaled_test)
        mlflow.log_metric("train_loss", evaled_train)

        plt.plot(history.history["loss"])
        plt.plot(history.history["val_loss"])
        plt.title('Model Loss')
        plt.ylabel('Loss')
        plt.xlabel('Epoch')
        plotname = f"plots/loss_plot_{run.info.run_id}.png"
        plt.savefig(plotname)
        plt.close()

        # Log the plot as an artifact
        mlflow.log_artifact(plotname)
        #model_path = "modles"
        #mlflow.tensorflow.log_model(tf_saved_model_dir=model_path, artifact_path="model")


2024/09/05 08:53:52 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/05 08:53:52 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4372/4372 [==============================] - 51s 11ms/step - loss: 9.1588e-04 - val_loss: 4.1626e-04
Epoch 2/1000
4372/4372 [==============================] - 49s 11ms/step - loss: 4.5243e-04 - val_loss: 4.7280e-04
Epoch 3/1000
4372/4372 [==============================] - 49s 11ms/step - loss: 4.2508e-04 - val_loss: 4.0776e-04
Epoch 4/1000
4372/4372 [==============================] - 50s 11ms/step - loss: 4.1357e-04 - val_loss: 4.0473e-04
Epoch 5/1000
4372/4372 [==============================] - 49s 11ms/step - loss: 4.0659e-04 - val_loss: 4.1138e-04
Epoch 6/1000
4372/4372 [==============================] - 49s 11ms/step - loss: 4.0217e-04 - val_loss: 4.0087e-04
Epoch 7/1000
4372/4372 [==============================] - 49s 11ms/step - loss: 3.9852e-04 - val_loss: 3.7526e-04
Epoch 8/1000
4372/4372 [==============================] - 48s 11ms/step - loss: 3.9809e-04 - val_loss: 4.0884e-04
Epoch 9/1000
4372/4372 [==============================] - 49s 11ms/step - loss: 3.9430e-

2024/09/05 09:16:39 WARNING mlflow.utils.autologging_utils: MLflow autologging encountered a warning: "/data/jmackin1/miniconda/envs/tf-gpu/lib/python3.10/site-packages/_distutils_hack/__init__.py:33: UserWarning: Setuptools is replacing distutils."


9698/9698 [==============================] - 25s 3ms/step - loss: 4.0991e-04


2024/09/05 09:20:41 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/05 09:20:41 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


...

Epoch 1/1000
4372/4372 [==============================] - 54s 12ms/step - loss: 0.0010 - val_loss: 5.5075e-04
Epoch 2/1000
4372/4372 [==============================] - 54s 12ms/step - loss: 5.8754e-04 - val_loss: 4.8538e-04
Epoch 3/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 5.1442e-04 - val_loss: 4.3890e-04
Epoch 4/1000
4372/4372 [==============================] - 52s 12ms/step - loss: 5.2832e-04 - val_loss: 4.9209e-04
Epoch 5/1000
4372/4372 [==============================] - 54s 12ms/step - loss: 5.5316e-04 - val_loss: 5.3245e-04
Epoch 6/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 5.4511e-04 - val_loss: 5.4057e-04
Epoch 7/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 5.3534e-04 - val_loss: 5.5677e-04
Epoch 8/1000
4372/4372 [==============================] - 52s 12ms/step - loss: 5.3066e-04 - val_loss: 5.5616e-04
Epoch 9/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 5.2819e-04 -

2024/09/06 09:50:54 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/06 09:50:54 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4372/4372 [==============================] - 55s 12ms/step - loss: 4.5180e-04 - val_loss: 3.2567e-04
Epoch 2/1000
4372/4372 [==============================] - 54s 12ms/step - loss: 2.9918e-04 - val_loss: 2.9138e-04
Epoch 3/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 2.8043e-04 - val_loss: 2.8108e-04
Epoch 4/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 2.7293e-04 - val_loss: 2.8264e-04
Epoch 5/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 2.6754e-04 - val_loss: 2.8982e-04
Epoch 6/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 2.6412e-04 - val_loss: 2.8743e-04
Epoch 7/1000
4372/4372 [==============================] - 54s 12ms/step - loss: 2.6138e-04 - val_loss: 2.8001e-04
Epoch 8/1000
4372/4372 [==============================] - 54s 12ms/step - loss: 2.5950e-04 - val_loss: 2.7929e-04
Epoch 9/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 2.5701e-

2024/09/06 10:20:25 WARNING mlflow.tensorflow: Unrecognized dataset type <class 'dict'>. Dataset logging skipped.
2024/09/06 10:20:25 WARNING mlflow.tensorflow: Failed to log training dataset information to MLflow Tracking. Reason: 'tensorflow.python.framework.ops.EagerTensor' object has no attribute 'flatten'


Epoch 1/1000
4372/4372 [==============================] - 53s 12ms/step - loss: 0.0028 - val_loss: 3.8746e-04
Epoch 2/1000
4372/4372 [==============================] - 51s 12ms/step - loss: 4.4103e-04 - val_loss: 3.7035e-04
Epoch 3/1000
4372/4372 [==============================] - 51s 12ms/step - loss: 4.2737e-04 - val_loss: 3.5707e-04
Epoch 4/1000
4372/4372 [==============================] - 51s 12ms/step - loss: 4.2016e-04 - val_loss: 3.5805e-04
Epoch 5/1000
4372/4372 [==============================] - 50s 11ms/step - loss: 4.1625e-04 - val_loss: 3.5144e-04
Epoch 6/1000
4372/4372 [==============================] - 50s 11ms/step - loss: 4.1511e-04 - val_loss: 3.5013e-04
Epoch 7/1000
4372/4372 [==============================] - 50s 11ms/step - loss: 4.1419e-04 - val_loss: 3.4689e-04
Epoch 8/1000
4372/4372 [==============================] - 49s 11ms/step - loss: 4.1371e-04 - val_loss: 3.4227e-04
Epoch 9/1000
4372/4372 [==============================] - 50s 12ms/step - loss: 4.1328e-04 -